In [9]:
# !pip install torch transformers datasets tqdm numpy matplotlib
!pip install peft
# !pip install git+https://github.com/joshuacnf/Ctrl-G

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import copy

In [ ]:
from peft import LoraConfig, get_peft_model

# Load model and tokenizer
# model_name = "gpt2"  # Using a small model for demonstration
# use gp2 large
# model_name = "gpt2-large"
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
 
# Create a reference model (pre-RL state)
ref_model = copy.deepcopy(model)

# apply lora

# config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     lora_dropout=0.05,
#     target_modules=[
#         "q_proj",
#         "gate_proj",
#         "v_proj",
#         "o_proj",
#         "k_proj",
#         "up_proj",
#         "down_proj"
#     ],
#     bias="none",
#     task_type="CAUSAL_LM",
# )
# gemma config
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)


# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
ref_model.to(device)

# print trainable parameters


print(f"Using device: {device}")

In [9]:
# Load HMM model and constraints
import ctrlg
# Load HMM model and constraints
import ctrlg
from huggingface_hub import snapshot_download

tokenizer = AutoTokenizer.from_pretrained("gpt2")


HMM_MODEL_PATH = f'ctrlg/hmm_gpt2-large_common-gen_4096' # alternatively 'ctrlg/hmm_gpt2-large_common-gen_32768' for better quality
# HMM_MODEL_PATH = snapshot_download(
#     repo_id="gwenweng/gemma",
#     local_dir="models/gemma"
# )
hmm_model = ctrlg.HMM.from_pretrained(HMM_MODEL_PATH).to(device)
vocab_size = hmm_model.vocab_size
eos_token_id = hmm_model.eos_token_id

def get_dfa_model(prompt_ids, keyphrases=[[' ']], suffix_ids=None, min_new_tokens=5, max_new_tokens=32):

    ##################################### prefix, suffix, prompt #####################################
    prefix = '' # generate text starting with nothing
    suffix = '.<|endoftext|>' # generate text ending with '<|endoftext|>'; a suffix must end with the eos token
    # prompt = '<|endoftext|>' # prompt the base model with the '<|endoftext|>' token

    prefix_ids = tokenizer.encode(prefix)
    if suffix_ids is None:
        suffix_ids = tokenizer.encode(suffix)
    # prompt_ids = tokenizer.encode(prompt)
    ##################################### prefix, suffix, prompt #####################################


    ##################################### DFA Construction #####################################
    # ac_builder constructs a DFA representing the constraint that (at least) 
    # one the patterns must appear; a pattern is a sequence of token ids
    ac_builder = ctrlg.AhoCorasickBuilder(vocab_size)

    dfa_graphs = []

    # constraint 1:
    # one of ' riding a bike', ' ride bikes', ' rides a bike', ' biking', ' bikes' has to appear
    # AND one of ' park', ' beach' has to appear
    # keyphrases = [[' riding a bike', ' ride bikes', ' rides a bike', ' biking', ' bikes'],
    #             [' park', ' beach']]
    for keyphrase in keyphrases:
        patterns = [tokenizer.encode(x) for x in keyphrase]
        dfa_graphs.append(ac_builder.build(patterns))

    # taking the intersection of the DFAs, i.e., "logical and" of the constraints.
    # This function also minimizes the constructed DFA, which is mainly CPU-based operations;
    # Due to its pure python implemenation, DFA minimization can be slow for complex constraints
    dfa_graph = ctrlg.DFA_prod(dfa_graphs, mode='intersection')

    # compile the dfa_graph for efficient GPU execution
    dfa_model = ctrlg.DFAModel(dfa_graph, vocab_size).to(device)
    ##################################### DFA Construction #####################################


    ##################################### token length #####################################
    # specify the min_new_tokens and max_new_tokens to be generated (excluding
    # the prefix and suffix) make sure that the numbers here would not conflict
    # with the given constraint: e.g. ask the model to generate 10 words with
    # max_new_tokens = 8
    # min_new_tokens = 5
    # max_new_tokens = 32
    ##################################### token length #####################################

    # # DEBUG:
    # print(f"prompt_ids shape: {prompt_ids.shape}")  
    # print(f"prefix_ids shape: {prefix_ids.shape}")
    # print(f"suffix_ids shape: {suffix_ids.shape}")

    constraint_logits_processor = ctrlg.ConstraintLogitsProcessor(
        hmm_model, 
        dfa_model,
        min_new_tokens, 
        max_new_tokens,
        prompt_ids, 
        prefix_ids=prefix_ids, 
        suffix_ids=suffix_ids
    )

    return constraint_logits_processor


# test
get_dfa_model(prompt_ids=torch.randint(0, tokenizer.vocab_size, (1, 10)))


InductorError: CppCompileError: C++ compile error

Command:
clang++ /var/folders/r_/d81gvkws1n5fg2_7mb57cwzh0000gn/T/torchinductor_marawangamal/hg/chggtlcta6pvwexwktea6pkszd3w5u6myszoiy2l7sulr5dpbk3h.main.cpp -D TORCH_INDUCTOR_CPP_WRAPPER -D STANDALONE_TORCH_HEADER -D C10_USING_CUSTOM_GENERATED_MACROS -D CPU_CAPABILITY_NEON -D AT_BUILD_ARM_VEC256_WITH_SLEEF -O3 -DNDEBUG -fno-trapping-math -funsafe-math-optimizations -ffinite-math-only -fno-signed-zeros -fno-math-errno -fno-finite-math-only -fno-unsafe-math-optimizations -ffp-contract=off -shared -fPIC -undefined dynamic_lookup -Wall -std=c++17 -Wno-unused-variable -Wno-unknown-pragmas -Werror=ignored-optimization-argument -Xclang -fopenmp -include /var/folders/r_/d81gvkws1n5fg2_7mb57cwzh0000gn/T/torchinductor_marawangamal/precompiled_headers/c62bfr35myvt7jik6nuf236zkm2n42zpgvjfrvmt26eeh234b7cl.h -I/opt/homebrew/opt/python@3.13/Frameworks/Python.framework/Versions/3.13/include/python3.13 -I/Users/marawangamal/Documents/github/ctrl-rlvr/

In [6]:

from datasets import load_dataset
gsm8k = load_dataset("openai/gsm8k", "main")

def load_gsm8k(n=100):
    data = gsm8k["train"].select(range(n))

    formatted = []
    for item in data:
        problem = item["question"]
        solution = item["answer"]  # GSM8K solutions are full step-by-step
        formatted.append({
            "problem": problem,
            "solution": solution,
            "answer": extract_answer_gsm8k(solution)
        })
    return formatted

def generate_math_problems(num_problems=100):
    problems = []
    for _ in range(num_problems):
        a = random.randint(1, 10)
        b = random.randint(1, 10)
        op = random.choice(['+', '-', '*'])
         
        if op == '+':
            answer = a + b
        elif op == '-':
            answer = a - b
        else:
            answer = a * b
             
        problem = f"What is {a} {op} {b}?"
        solution = f"The answer is {answer}."
        problems.append({"problem": problem, "solution": solution, "answer": answer})
     
    return problems
 
# math_problems = generate_math_problems(100)
math_problems = load_gsm8k(1000)

{'problem': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'solution': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [7]:
def extract_answer(text):
    """Extract the numerical answer from the text."""
    try:
        # Basic extraction - look for numbers after "answer is"
        if "answer is" in text:
            answer_part = text.split("answer is")[1].strip()
            # Extract the first number
            for word in answer_part.split():
                word = word.strip('.,')
                if word.isdigit() or (word[0] == '-' and word[1:].isdigit()):
                    return int(word)
    except:
        pass
    return None


def extract_answer_gsm8k(text):
    """Extract the numerical answer from the text."""
    try:
        # Basic extraction - parse after ####
        if "####" in text:
            answer_part = text.split("####")[1].strip()
            # Extract the first number
            for word in answer_part.split():
                word = word.strip('.,')
                if word.isdigit() or (word[0] == '-' and word[1:].isdigit()):
                    return int(word)
    except:
        pass
    return None

 
def compute_reward(response, correct_answer):
    """Compute reward based on correctness."""
    extracted = extract_answer(response)
    if extracted is not None and extracted == correct_answer:
        return 1.0  # Correct answer
    return 0.0  # Incorrect answer


72

In [ ]:
from transformers import LogitsProcessorList

def get_logprobs(model, input_ids, attention_mask):
    """Get log probabilities for each token."""
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits[:, :-1, :]  # Remove last position
         
        # Get log probabilities
        log_probs = F.log_softmax(logits, dim=-1)
         
        # Get the log probability of the actual next token
        target_ids = input_ids[:, 1:]  # Shift right
        gathered_logprobs = log_probs.gather(dim=-1, index=target_ids.unsqueeze(-1)).squeeze(-1)
         
        # Mask out padding
        masked_logprobs = gathered_logprobs * attention_mask[:, 1:].float()
         
        return masked_logprobs
 
def compute_advantages(rewards, group_rewards):
    """Compute advantages using the group baseline."""
    mean_reward = np.mean(group_rewards)
    std_reward = np.std(group_rewards) + 1e-8  # Add small epsilon to avoid division by zero
     
    # Normalize rewards
    advantages = (rewards - mean_reward) / std_reward
    return advantages
 
def grpo_loss(current_logprobs, old_logprobs, ref_logprobs, advantages, clip_epsilon=0.2, kl_coef=0.1):
    """Compute the GRPO loss."""
    # Compute probability ratio
    ratio = torch.exp(current_logprobs - old_logprobs)
     
    # Compute clipped objective
    clipped_ratio = torch.clamp(ratio, 1 - clip_epsilon, 1 + clip_epsilon)
    ppo_obj = torch.min(ratio * advantages, clipped_ratio * advantages)
     
    # Compute KL divergence term
    kl_div = old_logprobs - ref_logprobs
     
    # Compute final loss
    loss = -ppo_obj.mean() + kl_coef * kl_div.mean()
     
    return loss
 
def train_grpo(model, ref_model, problem, correct_answer, solution, group_size=4, max_length=50):
    """Train the model using GRPO on a single problem."""
    tokenizer.pad_token = tokenizer.eos_token
     
    # Generate responses for the group
    group_responses = []
    group_rewards = []
     
    # Create a batch of identical prompts
    prompt = f"Please solve the following math problem: {problem}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

    # HPs
    beam_size = 32
    min_new_tokens = 6
    max_new_tokens = 32
     
    # Generate group responses
    for _ in range(group_size):
        with torch.no_grad():
            output_ids = model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_length=max_length + inputs.input_ids.shape[1],
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7
            )

            # suffix_ids = tokenizer.encode(solution)
            # constraint_logits_processor = get_dfa_model(prompt_ids=inputs.input_ids[0].tolist(), suffix_ids=suffix_ids)
            # output_ids = model.generate(
            #     input_ids=inputs.input_ids, do_sample=False, length_penalty=0.2,
            #     num_beams=beam_size, num_return_sequences=beam_size,
            #     min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
            #     logits_processor=LogitsProcessorList([constraint_logits_processor]),
            #     pad_token_id=tokenizer.eos_token_id,
            # )
             
            # Get the generated response
            response = tokenizer.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
            reward = compute_reward(response, correct_answer)
             
            group_responses.append(response)
            group_rewards.append(reward)
     
    # Compute advantages
    advantages = compute_advantages(group_rewards, group_rewards)
     
    # Optimize the model for each response
    optimizer = Adam(model.parameters(), lr=1e-5)
     
    for i in range(group_size):
        # Get the full sequence
        full_sequence = tokenizer(prompt + group_responses[i], return_tensors="pt").to(device)
         
        # Get old logprobs
        old_logprobs = get_logprobs(model, full_sequence.input_ids, full_sequence.attention_mask)
         
        # Get reference logprobs
        ref_logprobs = get_logprobs(ref_model, full_sequence.input_ids, full_sequence.attention_mask)
         
        # Forward pass
        outputs = model(input_ids=full_sequence.input_ids, attention_mask=full_sequence.attention_mask)
        logits = outputs.logits[:, :-1, :]
         
        # Compute new logprobs
        log_probs = F.log_softmax(logits, dim=-1)
        target_ids = full_sequence.input_ids[:, 1:]
        current_logprobs = log_probs.gather(dim=-1, index=target_ids.unsqueeze(-1)).squeeze(-1)
         
        # Mask out padding
        mask = full_sequence.attention_mask[:, 1:].float()
        current_logprobs = current_logprobs * mask
         
        # Compute loss
        advantage = torch.tensor([advantages[i]]).to(device).expand_as(current_logprobs)
        loss = grpo_loss(current_logprobs, old_logprobs, ref_logprobs, advantage)
         
        # Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
         
    return np.mean(group_rewards)

In [ ]:
def evaluate_model(model, math_problems, num_samples=10):
    """Evaluate the model on a subset of math problems."""
    correct = 0
    samples = random.sample(math_problems, min(num_samples, len(math_problems)))
     
    for problem in samples:
        prompt = f"Please solve the following math problem: {problem['problem']}"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
         
        with torch.no_grad():
            output_ids = model.generate(
                inputs.input_ids,
                max_length=50 + inputs.input_ids.shape[1],
                pad_token_id=tokenizer.eos_token_id
            )
             
            response = tokenizer.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
            reward = compute_reward(response, problem['answer'])
            correct += reward
     
    return correct / len(samples)
 
# Training loop
epochs = 3
training_problems = math_problems[:80]  # Use 80% for training
eval_problems = math_problems[80:]      # Use 20% for evaluation
 
performance_history = []
 
for epoch in range(epochs):
    epoch_rewards = []
     
    for problem in tqdm(training_problems, desc=f"Epoch {epoch+1}/{epochs}"):
        reward = train_grpo(model, ref_model, problem['problem'], problem['answer'], problem['solution'])
        epoch_rewards.append(reward)
     
    # Evaluate model
    accuracy = evaluate_model(model, eval_problems)
    performance_history.append(accuracy)
     
    print(f"Epoch {epoch+1}/{epochs} - Average Reward: {np.mean(epoch_rewards):.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/3:   0%|          | 0/80 [00:00<?, ?it/s]W1116 21:39:21.604000 84096 torch/_dynamo/convert_frame.py:1358] [0/8] torch._dynamo hit config.recompile_limit (8)
W1116 21:39:21.604000 84096 torch/_dynamo/convert_frame.py:1358] [0/8]    function: 'matmul_a_logb' (/Users/marawangamal/Documents/github/ctrl-rlvr/.venv/lib/python3.13/site-packages/ctrlg/utils.py:40)
W1116 21:39:21.604000 84096 torch/_dynamo/convert_frame.py:1358] [0/8]    last reason: 0/3: tensor 'A' size mismatch at index 0. expected 4096, actual 3. Guard failed on a parameter, consider using torch._dynamo.config.force_parameter_static_shapes = False to allow dynamism on parameters.
W1116 21:39:21.604000 84096 torch/_dynamo/convert_frame.py:1358] [0/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W1116 21:39:21.604000 84096 torch/_dynamo/convert_frame.py:1358] [0/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html
Epoch 1/3:   1%|▏         | 1/80 [

In [25]:
from transformers import LogitsProcessorList

max_new_tokens = 32
min_new_tokens = 6
beam_size = 32

def test_model(model, num_problems=5):
    """Test the model on new math problems."""
    test_problems = generate_math_problems(num_problems)

     
    for problem in test_problems:
        prompt = f"Please solve the following math problem: {problem['problem']}"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
         
        with torch.no_grad():
            # output_ids = model.generate(
            #     inputs.input_ids,
            #     max_length=50 + inputs.input_ids.shape[1],
            #     pad_token_id=tokenizer.eos_token_id,
            # )

            suffix_ids = tokenizer.encode(problem['solution'])
            constraint_logits_processor = get_dfa_model(prompt_ids=inputs.input_ids[0].tolist(), suffix_ids=suffix_ids)
            output_ids = model.generate(
                input_ids=inputs.input_ids, do_sample=False, length_penalty=0.2,
                num_beams=beam_size, num_return_sequences=beam_size,
                min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
                logits_processor=LogitsProcessorList([constraint_logits_processor]),
                pad_token_id=tokenizer.eos_token_id,
            )
             
            response = tokenizer.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
            reward = compute_reward(response, problem['answer'])
             
            print(f"Problem: {problem['problem']}")
            print(f"Model's response: {repr(response)}")
            print(f"Correct answer: {problem['answer']}")
            print(f"Reward: {reward}")
            print("-" * 50)
 
# Test the model
test_model(model)


Problem: What is 8 - 1?
Model's response: ' \xa0Answer: 8\nThe answer is 7.\nThe answer is 7.\nThe answer is 7.\nThe answer is 7.\nThe answer'
Correct answer: 7
Reward: 1.0
--------------------------------------------------
Problem: What is 10 + 5?
Model's response: '\n\n10 + 5 = 20\n\n20 + 5 = 30\n\n30 + 5 = 40\n\n40 + 5 = 50\n\n50 '
Correct answer: 15
Reward: 0.0
--------------------------------------------------
Problem: What is 1 - 7?
Model's response: '\n\nAnswer: ________\n\nThe answer is -6.\n\nThe answer is -6.\n\nThe answer is -6.\n\n'
Correct answer: -6
Reward: 1.0
--------------------------------------------------


KeyboardInterrupt: 

In [14]:
test_problems = generate_math_problems(5)
for problem in test_problems:
    prompt = f"Please solve the following math problem: {problem['problem']}"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    break
suffix_ids = tokenizer.encode(problem['solution'])
print(suffix_ids)

[464, 3280, 318, 860, 13]


In [ ]:
output_ids = model.generate(
        input_ids=inputs.input_ids, do_sample=False, length_penalty=0.2,
    num_beams=beam_size, num_return_sequences=beam_size,
    min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
    logits_processor=LogitsProcessorList([constraint_logits_processor]),
    pad_token_id=tokenizer.eos_token_id,
)

In [33]:
inputs.input_ids[0].tolist()

[5492, 8494, 262, 1708, 10688, 1917, 25, 1867, 318, 352, 532, 352, 30]

In [ ]:


constraint_logits_processor = get_dfa_model(prompt_ids=inputs.input_ids[0].tolist(), keyphrases=[[problem['solution']]])

output_ids = model.generate(
    input_ids=inputs.input_ids, do_sample=False, length_penalty=0.2,
    num_beams=beam_size, num_return_sequences=beam_size,
    min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
    logits_processor=LogitsProcessorList([constraint_logits_processor]),
    pad_token_id=tokenizer.eos_token_id,
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
tokenizer.decode(output_ids[0], skip_special_tokens=True)

'Please solve the following math problem: What is 2 * 5?\n\nThe answer is 2.'

In [39]:
problem

{'problem': 'What is 2 * 5?', 'solution': 'The answer is 10.', 'answer': 10}